In [116]:
using ModelingToolkit, OrdinaryDiffEq
using ModelingToolkit: t_nounits as t, D_nounits as D

using CoolProp.CoolProp: PropsSI
using BenchmarkTools
using Plots

In [117]:
global fluid_name
fluid_name = "Parahydrogen"

"Parahydrogen"

In [118]:
# First define the temperature calculation function
function calc_T(h, p)
    return PropsSI("T", "H", h, "P", p, fluid_name)
end

# Register the function for symbolic use
@register_symbolic calc_T(h::Real, p::Real)


@connector FluidPort begin
    @variables begin
        h(t), [guess=0.0]
        p(t), [guess=0.0]
        ṁ(t), [guess=0.0, connect = Flow]
    end
end

@mtkmodel Source begin
    @parameters begin
        h_source
        p_source
        ṁ_source
    end

    @components begin
        outlet = FluidPort()
    end

    @equations begin
        outlet.h ~ h_source
        outlet.p ~ p_source
        outlet.ṁ ~ ṁ_source
    end
end

@mtkmodel Sink begin

    @components begin
        inlet = FluidPort()
    end

end

@mtkmodel Heater begin

    @components begin
        inlet = FluidPort()
        outlet = FluidPort()
    end

    @parameters begin
        T_Al            
        dL            
        UA_over_dL
    end

    @variables begin
        T(t)
        Q̇(t)
    end

    @equations begin
        T ~ calc_T(inlet.h, inlet.p)
        Q̇ ~ UA_over_dL*dL*(T_Al - T)
        outlet.h ~ inlet.h + Q̇/outlet.ṁ
        outlet.p ~ inlet.p
        outlet.ṁ + inlet.ṁ ~ 0 
    end
end

@mtkmodel System begin
    @parameters begin
        h_source
        p_source
        ṁ_source
        T_Al      
        dL            
        UA_over_dL
    end

    @components begin
        source = Source(; h_source=h_source, p_source=p_source, ṁ_source=ṁ_source)
        heater = Heater(; T_Al=T_Al, dL=dL, UA_over_dL=UA_over_dL)
        sink = Sink()
    end

    @equations begin
        connect(source.outlet, heater.inlet)
        connect(heater.outlet, sink.inlet)
    end
end;

In [119]:
## Inputs
# Fluid parameters
ṁ = 10e-3            # kg/s
T_in = 70               # K
p_in = 350e5           # Pa
h_in = PropsSI("H", "T", T_in, "P", p_in, fluid_name);

# Wall parameters
T_Al = 50 + 273.15      # K
L = 10.0                 # m
dL = 0.1                # m
UA_over_dL = 100.0;

In [120]:
@mtkbuild sys = System(; h_source=h_in, p_source=p_in, ṁ_source=ṁ, T_Al=T_Al, dL=dL, UA_over_dL=UA_over_dL)

┌ Warning: outlet contains 1 flow variables, yet 2 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit C:\Users\matth\.julia\packages\ModelingToolkit\aau6A\src\systems\connectors.jl:54
┌ Warning: inlet contains 1 flow variables, yet 2 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit C:\Users\matth\.julia\packages\ModelingToolkit\aau6A\src\systems\connectors.jl:54
┌ Warning: outlet contains 1 flow variables, yet 2 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit C:\Users\mat

Model sys:
Parameters (12): see parameters(sys)
  source₊ṁ_source [defaults to ṁ_source]
  source₊p_source [defaults to p_source]
  ṁ_source [defaults to 0.01]
  heater₊T_Al [defaults to T_Al]
  ⋮
Observed (14): see observed(sys)

In [121]:
prob = ODEProblem(sys, [], (0, 10.0))
sol = solve(prob, Tsit5());

In [122]:
println(h_in)
println(sol[sys.heater.outlet.h][end])
println(sol[sys.heater.T][end])

872335.6641158205
1.1254856641163882e6
69.99999999943222
